In [34]:
import pyvirtualdisplay
import gym
import pickle

In [35]:
with open("trajectory04-27-2022,16-52.pkl", "rb") as trajectoryfile:
    trajectories = pickle.load(trajectoryfile)

trajectories[0]

(array([[[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[228, 111, 111],
         [228, 111, 111],
         [228, 111, 111],
         ...,
         [228, 111, 111],
         [228, 111, 111],
         [228, 111, 111]],
 
        [[228, 111, 111],
         [228, 111, 111],
         [228, 111, 111],
         ...,
         [228, 111, 111],
         [228, 111, 111],
         [228, 111, 111]],
 
        ...,
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
  

In [36]:
import numpy as np

from tf_agents.trajectories import Trajectory
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.environments.atari_preprocessing import AtariPreprocessing
from tf_agents.environments.atari_wrappers import FrameStack4


env_name = 'ALE/MsPacman-v5'

env = gym.make(env_name)

init_step = env.reset()
adict = env.unwrapped.get_keys_to_action()
print(adict)

train_py_env = suite_gym.load(env_name, 
                     gym_env_wrappers=[AtariPreprocessing, FrameStack4])
eval_py_env = suite_gym.load(env_name, 
                     gym_env_wrappers=[AtariPreprocessing, FrameStack4])

init_step = train_py_env.reset()

trajectory_list = []
for tuple in trajectories:
    new_traj = Trajectory(step_type=init_step.step_type, 
        observation=tuple[0], 
        action=tuple[2],
        policy_info=tuple[5],
        next_step_type=init_step.step_type,
        reward=tuple[3],
        discount=1.0)
    trajectory_list.append(new_traj)


{(None,): 0, (32,): 1, (119,): 2, (100,): 3, (97,): 4, (115,): 5, (100, 119): 6, (97, 119): 7, (100, 115): 8, (97, 115): 9, (32, 119): 10, (32, 100): 11, (32, 97): 12, (32, 115): 13, (32, 100, 119): 14, (32, 97, 119): 15, (32, 100, 115): 16, (32, 97, 115): 17}


In [37]:
import tensorflow as tf

In [38]:
np.shape(trajectories[0][0])

(210, 160, 3)

In [39]:
traj_data_spec = (
    tf.TensorSpec([210, 160, 3], tf.float32, 'observation'),
    tf.TensorSpec([1], tf.float32, 'action'),
    tf.TensorSpec([1], tf.float32, 'reward'),
    tf.TensorSpec([1], tf.float32, 'done'),
    (
        tf.TensorSpec([1], tf.float32, 'lives'),
        tf.TensorSpec([1], tf.float32, 'episode_frames_number'),
        tf.TensorSpec([1], tf.float32, 'frame_number')
    )
)

tensor_traj_list = []

for traj in trajectories:
    done_val = 0.0
    if traj[4]:
        done_val = 1.0

    interim = (
        tf.constant(traj[0], dtype = tf.float32, shape = (210,160,3), name ='observation'),
        tf.constant(traj[2], dtype = tf.float32, shape = (1), name = 'action'),
        tf.constant(traj[3], dtype = tf.float32, shape = (1), name = 'reward'),
        tf.constant(done_val, dtype = tf.float32, shape = (1), name = 'done'),
        (
            tf.constant(traj[5]['lives'], dtype = tf.float32, shape = (1), name = 'lives'),
            tf.constant(traj[5]['episode_frame_number'], dtype = tf.float32, shape = (1), name = 'episode_frame_number'),
            tf.constant(traj[5]['frame_number'], dtype = tf.float32, shape = (1), name = 'frame_number')
        )
    )

    #     interim = (
    #     np.array(traj[0]),
    #     np.array(traj[2]),
    #     np.array(traj[3]),
    #     np.array(done_val),
    #     (
    #         np.array(traj[5]['lives']),
    #         np.array(traj[5]['episode_frame_number']),
    #         np.array(traj[5]['frame_number'])
    #     )
    # )

    tensor_traj_list.append(interim)
print(np.shape(tensor_traj_list))

(2126, 5)


In [42]:
from tf_agents.specs import ArraySpec

# traj_data_spec = (
#     ArraySpec([210, 160, 3], np.float32, 'observation'),
#     ArraySpec([1], np.float32, 'action'),
#     ArraySpec([1], np.float32, 'reward'),
#     ArraySpec([1], np.float32, 'done'),
#     (
#         ArraySpec([1], np.float32, 'lives'),
#         ArraySpec([1], np.float32, 'episode_frames_number'),
#         ArraySpec([1], np.float32, 'frane')
#     )
# )

traj_data_spec = (
    tf.TensorSpec([210, 160, 3], tf.float32, 'observation'),
    tf.TensorSpec([1], tf.float32, 'action'),
    tf.TensorSpec([1], tf.float32, 'reward'),
    tf.TensorSpec([1], tf.float32, 'done'),
    (
        tf.TensorSpec([1], tf.float32, 'lives'),
        tf.TensorSpec([1], tf.float32, 'episode_frames_number'),
        tf.TensorSpec([1], tf.float32, 'frane')
    )
)

In [43]:
from tf_agents.replay_buffers import TFUniformReplayBuffer

replay_buffer = TFUniformReplayBuffer(
    traj_data_spec,
    batch_size=2, 
    max_length = 1000
)

In [49]:
batch_size = 2

to_batch = tf.nest.map_structure(lambda t: tf.stack([t] * batch_size), tensor_traj_list)
for element in to_batch:
    replay_buffer.add_batch(element)

In [54]:
print(replay_buffer.get_next())

((<tf.Tensor: shape=(210, 160, 3), dtype=float32, numpy=
array([[[  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]],

       [[228., 111., 111.],
        [228., 111., 111.],
        [228., 111., 111.],
        ...,
        [228., 111., 111.],
        [228., 111., 111.],
        [228., 111., 111.]],

       [[228., 111., 111.],
        [228., 111., 111.],
        [228., 111., 111.],
        ...,
        [228., 111., 111.],
        [228., 111., 111.],
        [228., 111., 111.]],

       ...,

       [[  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]],

       [[  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]],

       [[  0., 